# LAMDA-SSL による半教師あり学習の実装

## GPU ランタイムの有効化
1. Google Colaboratory のメニューから「ランタイム」をクリック
2. 「ランタイムのタイプを変更」をクリック
3. ハードウェアアクセラレータから「T4 GPU」をクリック
4. 「保存」をクリック
5. コードを上から順に実行する

## ライブラリのインストール

In [1]:
!pip install -q LAMDA-SSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.3 MB/s eta 0:00:00


## ライブラリのインポート

In [2]:
# LAMDA-SSL は学習〜評価をサポートする一連のクラスが用意されている

# データ拡張に関するもの
from LAMDA_SSL.Augmentation.Vision.RandomHorizontalFlip import RandomHorizontalFlip
from LAMDA_SSL.Augmentation.Vision.RandomCrop import RandomCrop

# データセットに関するもの
from LAMDA_SSL.Dataset.Vision.CIFAR10 import CIFAR10
from LAMDA_SSL.Dataset.LabeledDataset import LabeledDataset
from LAMDA_SSL.Dataset.UnlabeledDataset import UnlabeledDataset
from LAMDA_SSL.Dataloader.UnlabeledDataloader import UnlabeledDataLoader
from LAMDA_SSL.Dataloader.LabeledDataloader import LabeledDataLoader

# 学習に関するもの
from LAMDA_SSL.Opitimizer.SGD import SGD
from LAMDA_SSL.Scheduler.CosineAnnealingLR import CosineAnnealingLR

# モデルに関するもの
from LAMDA_SSL.Network.WideResNet import WideResNet

# 半教師あり学習手法に関するもの
from LAMDA_SSL.Algorithm.Classification.MeanTeacher import MeanTeacher
from LAMDA_SSL.Sampler.RandomSampler import RandomSampler
from LAMDA_SSL.Sampler.SequentialSampler import SequentialSampler

# 評価に関するもの
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Evaluation.Classifier.Top_k_Accuracy import Top_k_Accurary
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix

# PyTorch
import torch

# scikit-learn
from sklearn.pipeline import Pipeline

In [3]:
# warning を非表示にする
import warnings
warnings.simplefilter('ignore')

## データセットの準備

In [4]:
# CIFAR-10 データセットをダウンロード
dataset = CIFAR10(
    root='.', # ダウンロードしたデータの保存場所
    labeled_size=4000, # 学習データのうちラベルありとする枚数
    stratified=True, # 分割時に各クラスが同じ比率になるように分ける
    shuffle=True, # 分割前にシャッフルを実行
    download=True, # データセットをダウンロードする
    default_transforms=True # 標準化
)

100%|██████████| 170498071/170498071 [00:04<00:00, 37024728.25it/s]


Extracting ./cifar-10-python.tar.gz to .


In [5]:
# ラベル付きの学習用データ
labeled_X = dataset.labeled_X # 画像
labeled_y = dataset.labeled_y # ラベル

# ラベルなしの学習用データ
unlabeled_X=dataset.unlabeled_X # 画像

# 検証用データ
valid_X=dataset.valid_X # 画像
valid_y=dataset.valid_y # ラベル

# テストデータ
test_X=dataset.test_X # 画像
test_y=dataset.test_y # ラベル

In [6]:
# PyTorch の Dataset クラスを作成

# ラベル付きの学習用データ
labeled_dataset = LabeledDataset(
    pre_transform=dataset.pre_transform, # 画像を PIL 形式で読み込む
    transform=dataset.transform, # PyTorch のテンソルへ変換し、標準化
)

# ラベルなしの学習用データ
unlabeled_dataset=UnlabeledDataset(
    pre_transform=dataset.pre_transform, # 画像を PIL 形式で読み込む
    transform=dataset.unlabeled_transform # PyTorch のテンソルへ変換し、標準化
)

# 検証用データ
valid_dataset=UnlabeledDataset(
    pre_transform=dataset.pre_transform, # 画像を PIL 形式で読み込む
    transform=dataset.valid_transform # PyTorch のテンソルへ変換し、標準化
)

# テストデータ
test_dataset=UnlabeledDataset(
    pre_transform=dataset.pre_transform, # 画像を PIL 形式で読み込む
    transform=dataset.test_transform # PyTorch のテンソルへ変換し、標準化
)

In [7]:
# サンプラー（データの取り出し方）クラスを作成

# ラベル付き学習用データのサンプラー
labeled_sampler=RandomSampler( # 毎回ランダムに取り出す
    replacement=True, # 取り出したデータは同じメモリ領域に格納
    num_samples=64*1000 # 1エポックあたりのデータ数（バッチサイズ×イテレーション数）
)

# ラベルなし学習用データのサンプラー
unlabeled_sampler=RandomSampler(replacement=True)

# 検証用データのサンプラー
valid_sampler=SequentialSampler() # データセットの順番通りに取り出す

# テストデータのサンプラー
test_sampler=SequentialSampler()

In [8]:
# PyTorch のデータローダーを作成

# ラベル付き学習用データのデータローダー
labeled_dataloader = LabeledDataLoader(
    batch_size=64, # バッチサイズ
    num_workers=2, # CPU 2 コアを使って前処理を並列化
)

# ラベルなし学習用データのデータローダー
unlabeled_dataloader = UnlabeledDataLoader(
    num_workers=2,
)

# 検証用データのデータローダー
valid_dataloader = UnlabeledDataLoader(batch_size=64,num_workers=2,drop_last=False)

# テストデータのデータローダー
test_dataloader = UnlabeledDataLoader(batch_size=64,num_workers=2,drop_last=False)

## 学習の設定

In [9]:
# データ拡張の設定
augmentation=Pipeline(
    [
        # ランダムに左右反転
        ('RandomHorizontalFlip',RandomHorizontalFlip()),
        # ランダムに切り取って、元画像と同じ大きさに拡大
         ('RandomCrop',RandomCrop(
            padding=0.125,
            padding_mode='reflect')
        ),
    ]
)

# 最適化アルゴリズム（今回は SGD）
optimizer=SGD(
    lr=0.03, # 学習率
    momentum=0.9, # モーメンタム
    nesterov=True # ネステロフの加速勾配法を有効化
)

# 学習率のスケジューリング
scheduler=CosineAnnealingLR(
    eta_min=0, # 学習率の最小値
    T_max=1000 # 1000 イテレーションでゼロになるように設定
)

# モデルの設定（今回は WideResNet を使用）
network=WideResNet(
    num_classes=10, # 出力クラス数
    depth=28, # 深さ
    widen_factor=2, # 幅
    drop_rate=0 # ドロップアウト率（0なのでドロップアウトなし）
)

# 評価指標
evaluation={
    # 正答率
    'accuracy':Accuracy(),
    # 適合率
    'precision':Precision(average='macro'),
    # 再現率
    'Recall':Recall(average='macro'),
    # F1スコア
    'F1':F1(average='macro'),
    # AUC
    'AUC':AUC(multi_class='ovo'),
    # 混同行列
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

# GPU デバイス情報の取得
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [10]:
# Mean Teacher モデルの設定
model = MeanTeacher(
    lambda_u=50, # 一貫性正則化に対する係数
    warmup=0.4, # 全イテレーションの中で、最初に生徒モデルだけを更新する割合
    mu=1, # エントロピー正則化の係数
    weight_decay=5e-4, # 荷重減衰の係数
    ema_decay=0.999, # 指数移動平均の係数
    epoch=1, # エポック数
    num_it_epoch=1000, # エポックあたりのイテレーション数
    num_it_total=1000, # 合計イテレーション数
    eval_it=100, # 評価を何イテレーションごとに行うか
    device=device, # デバイス情報
    labeled_dataset=labeled_dataset, # ラベル付き学習データのデータセット
    unlabeled_dataset=unlabeled_dataset, # ラベルなし学習データのデータセット
    valid_dataset=valid_dataset, # 検証データのデータセット
    test_dataset=test_dataset, # テストデータのデータセット
    labeled_sampler=labeled_sampler, # ラベル付き学習データのサンプラー
    unlabeled_sampler=unlabeled_sampler, # ラベルなし学習データのサンプラー
    valid_sampler=valid_sampler, # 検証データのサンプラー
    test_sampler=test_sampler, # テストデータのサンプラー
    labeled_dataloader=labeled_dataloader, # ラベル付き学習データのデータローダー
    unlabeled_dataloader=unlabeled_dataloader, # ラベルなし学習データのデータローダー
    valid_dataloader=valid_dataloader, # 検証データのデータローダー
    test_dataloader=test_dataloader, # テストデータのデータローダー
    augmentation=augmentation, # データ拡張
    network=network, # モデル
    optimizer=optimizer, # 最適化アルゴリズム
    scheduler=scheduler, # 学習率のスケジューラー
    evaluation=evaluation, # 評価指標
    verbose=False # 経過の表示（今回は表示しない）
)

# 学習を実行
model.fit(
    X=labeled_X, # ラベル付き学習データの画像
    y=labeled_y, # ラベル付き学習データのラベル
    unlabeled_X=unlabeled_X, # ラベルなし学習データの画像
    valid_X=valid_X, # 検証データの画像
    valid_y=valid_y # 検証データのラベル
)

MeanTeacher(device=device(type='cuda', index=0), eval_it=100,
            evaluation={'AUC': <LAMDA_SSL.Evaluation.Classifier.AUC.AUC object at 0x7edc0fe38bb0>,
                        'Confusion_matrix': <LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix.Confusion_Matrix object at 0x7edc0fe39240>,
                        'F1': <LAMDA_SSL.Evaluation.Classifier.F1.F1 object at 0x7edc0fe3a140>,
                        'Recall': <LAMDA_SSL.Evaluation.Cl...
            unlabeled_sampler=<LAMDA_SSL.Sampler.RandomSampler.RandomSampler object at 0x7edc0fe39120>,
            valid_dataloader=<LAMDA_SSL.Dataloader.UnlabeledDataloader.UnlabeledDataLoader object at 0x7edc0fe3a1a0>,
            valid_dataset=<LAMDA_SSL.Dataset.UnlabeledDataset.UnlabeledDataset object at 0x7edc0fe39930>,
            valid_sampler=<LAMDA_SSL.Sampler.SequentialSampler.SequentialSampler object at 0x7edc0fe39f00>)

## 学習後の評価

In [11]:
# テストデータに対して評価を実行
performance=model.evaluate(
    X=test_X, # テストデータの画像
    y=test_y # テストデータのラベル
)

# 予測クラスを取得
result=model.y_pred

# 予測クラスを表示
print(result)

# 評価指標を表示
print(performance)

[3 8 8 ... 3 2 7]
{'accuracy': 0.414, 'precision': 0.5550694268511983, 'Recall': 0.414, 'F1': 0.4158885728220902, 'AUC': 0.8845816222222221, 'Confusion_matrix': array([[0.355, 0.009, 0.29 , 0.036, 0.004, 0.   , 0.008, 0.008, 0.282,
        0.008],
       [0.072, 0.293, 0.111, 0.127, 0.004, 0.001, 0.016, 0.004, 0.172,
        0.2  ],
       [0.033, 0.001, 0.768, 0.099, 0.014, 0.021, 0.032, 0.004, 0.028,
        0.   ],
       [0.004, 0.   , 0.379, 0.47 , 0.026, 0.061, 0.041, 0.004, 0.014,
        0.001],
       [0.009, 0.   , 0.632, 0.115, 0.133, 0.013, 0.058, 0.012, 0.026,
        0.002],
       [0.002, 0.   , 0.357, 0.368, 0.034, 0.204, 0.016, 0.008, 0.011,
        0.   ],
       [0.001, 0.001, 0.408, 0.124, 0.021, 0.002, 0.438, 0.001, 0.004,
        0.   ],
       [0.004, 0.001, 0.321, 0.203, 0.116, 0.049, 0.008, 0.284, 0.007,
        0.007],
       [0.042, 0.01 , 0.117, 0.045, 0.004, 0.005, 0.008, 0.003, 0.759,
        0.007],
       [0.031, 0.04 , 0.102, 0.149, 0.015, 0.   , 0.022,

In [12]:
print(result)

print(performance)

[3 8 8 ... 3 2 7]
{'accuracy': 0.414, 'precision': 0.5550694268511983, 'Recall': 0.414, 'F1': 0.4158885728220902, 'AUC': 0.8845816222222221, 'Confusion_matrix': array([[0.355, 0.009, 0.29 , 0.036, 0.004, 0.   , 0.008, 0.008, 0.282,
        0.008],
       [0.072, 0.293, 0.111, 0.127, 0.004, 0.001, 0.016, 0.004, 0.172,
        0.2  ],
       [0.033, 0.001, 0.768, 0.099, 0.014, 0.021, 0.032, 0.004, 0.028,
        0.   ],
       [0.004, 0.   , 0.379, 0.47 , 0.026, 0.061, 0.041, 0.004, 0.014,
        0.001],
       [0.009, 0.   , 0.632, 0.115, 0.133, 0.013, 0.058, 0.012, 0.026,
        0.002],
       [0.002, 0.   , 0.357, 0.368, 0.034, 0.204, 0.016, 0.008, 0.011,
        0.   ],
       [0.001, 0.001, 0.408, 0.124, 0.021, 0.002, 0.438, 0.001, 0.004,
        0.   ],
       [0.004, 0.001, 0.321, 0.203, 0.116, 0.049, 0.008, 0.284, 0.007,
        0.007],
       [0.042, 0.01 , 0.117, 0.045, 0.004, 0.005, 0.008, 0.003, 0.759,
        0.007],
       [0.031, 0.04 , 0.102, 0.149, 0.015, 0.   , 0.022,